In [1]:
%%capture
!pip install unsloth
# Also get the latest nightly Unsloth!
!pip install --force-reinstall --no-cache-dir --no-deps git+https://github.com/unslothai/unsloth.git

In [2]:
!pip index versions unsloth

unsloth (2025.1.6)
Available versions: 2025.1.6, 2025.1.5, 2025.1.4, 2025.1.3, 2025.1.2, 2025.1.1, 2024.12.12, 2024.12.11, 2024.12.10, 2024.12.9, 2024.12.8, 2024.12.7, 2024.12.6, 2024.12.5, 2024.12.4, 2024.12.3, 2024.12.2, 2024.12.1, 2024.11.11, 2024.11.10, 2024.11.9, 2024.11.8, 2024.11.7, 2024.11.6, 2024.11.5, 2024.11.4, 2024.11.2, 2024.10.7, 2024.10.6, 2024.10.5, 2024.10.4, 2024.10.2, 2024.10.1, 2024.10.0, 2024.9.post4, 2024.9.post3, 2024.9.post2, 2024.9.post1, 2024.9, 2024.8
  INSTALLED: 2025.1.7
  LATEST:    2025.1.6


In [3]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 8192 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

# 4bit pre quantized models we support for 4x faster downloading + no OOMs.
fourbit_models = [
    "unsloth/Meta-Llama-3.1-8B-bnb-4bit",      # Llama-3.1 15 trillion tokens model 2x faster!
    "unsloth/Meta-Llama-3.1-8B-Instruct-bnb-4bit",
    "unsloth/Meta-Llama-3.1-70B-bnb-4bit",
    "unsloth/Meta-Llama-3.1-405B-bnb-4bit",    # We also uploaded 4bit for 405b!
    "unsloth/Mistral-Nemo-Base-2407-bnb-4bit", # New Mistral 12b 2x faster!
    "unsloth/Mistral-Nemo-Instruct-2407-bnb-4bit",
    "unsloth/mistral-7b-v0.3-bnb-4bit",        # Mistral v3 2x faster!
    "unsloth/mistral-7b-instruct-v0.3-bnb-4bit",
    "unsloth/Phi-3.5-mini-instruct",           # Phi-3.5 2x faster!
    "unsloth/Phi-3-medium-4k-instruct",
    "unsloth/gemma-2-9b-bnb-4bit",
    "unsloth/gemma-2-27b-bnb-4bit",            # Gemma 2x faster!
] # More models at https://huggingface.co/unsloth

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Meta-Llama-3.1-8B-Instruct-bnb-4bit",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.1.7: Fast Llama patching. Transformers: 4.48.1.
   \\   /|    GPU: Tesla T4. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu121. CUDA: 7.5. CUDA Toolkit: 12.1. Triton: 3.1.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post1. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/5.70G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/234 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/55.4k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/340 [00:00<?, ?B/s]

In [4]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Unsloth 2025.1.7 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


In [5]:
FinQA_prompt = """Below contains texts before table (pre-text), text after the table (post-text) and the table itself with a question that you must answer.

### Pre-text:
{}

### Table:
{}

### Post-text:
{}

### Question:
{}

### Response:
{}"""

EOS_TOKEN = tokenizer.eos_token # Must add EOS_TOKEN
def formatting_prompts_func(examples):
    
    pre_texts      = examples["pre_text"]
    tables         = examples["table"]
    post_texts     = examples["post_text"]
    programs       = examples["program_re"]
    questions      = examples["question"]
    answers        = examples["answer"]

    
    texts = []
    for pre_text, table, post_text, program, question, answer in zip(pre_texts, tables, post_texts, programs, questions, answers):
        # Must add EOS_TOKEN, otherwise your generation will go on forever!
        text = FinQA_prompt.format(pre_text, table, post_text, question, program + ' = ' + answer) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }
pass

from datasets import load_dataset
train = load_dataset("ibm/finqa", split = "train", trust_remote_code=True)
test = load_dataset("ibm/finqa", split = "test", trust_remote_code=True)
validation = load_dataset("ibm/finqa", split = "validation", trust_remote_code=True)
train = train.map(formatting_prompts_func, batched = True)
#test = test.map(formatting_prompts_func, batched = True)
#validation = validation.map(formatting_prompts_func, batched = True)

README.md:   0%|          | 0.00/1.00k [00:00<?, ?B/s]

finqa.py:   0%|          | 0.00/4.49k [00:00<?, ?B/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/6251 [00:00<?, ? examples/s]

In [6]:
len(train)

6251

In [ ]:
FastLanguageModel.for_inference(model)

samp = ["""

### User prompt:
Hi, what are you?

### responce:
"""]

test_input = tokenizer(samp, return_tensors = "pt").to("cuda")
outputs = model.generate(**test_input, max_new_tokens = 64, use_cache = True, temperature = 1e-10)
tokenizer.batch_decode(outputs)

In [7]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = train,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 3,
        #remove_unused_columns=False,
        gradient_accumulation_steps = 6,
        warmup_steps = 5,
        num_train_epochs = 1, # Set this for 1 full training run.
        max_steps = 350,
        learning_rate = 1e-3,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
        report_to = "none", # Use this for WandB etc
    ),
)

Map (num_proc=2):   0%|          | 0/6251 [00:00<?, ? examples/s]

# Show current memory stats

In [8]:
#@title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = Tesla T4. Max memory = 14.741 GB.
5.496 GB of memory reserved.


In [ ]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 6,251 | Num Epochs = 2
O^O/ \_/ \    Batch size per device = 3 | Gradient Accumulation steps = 6
\        /    Total batch size = 18 | Total steps = 350
 "-____-"     Number of trainable parameters = 41,943,040


Step,Training Loss
1,1.706600
2,1.753400
3,1.799400
4,1.610200
5,1.553800
6,1.475200
7,1.381400
8,1.399700
9,1.457900
10,1.331500


In [ ]:
#@title Show final memory and time stats
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory         /max_memory*100, 3)
lora_percentage = round(used_memory_for_lora/max_memory*100, 3)
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training.")
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

In [ ]:
test = load_dataset("ibm/finqa", split = "test", trust_remote_code=True)

def formatting_prompts_test(examples):
    
    pre_texts      = examples["pre_text"]
    tables         = examples["table"]
    post_texts     = examples["post_text"]
    programs = examples["program_re"]
    questions      = examples["question"]
    answers        = examples["answer"]

    
    texts = []
    responses = []
    for pre_text, table, post_text, program, question, answer in zip(pre_texts, tables, post_texts, programs, questions, answers):
        # Must add EOS_TOKEN, otherwise your generation will go on forever!
        text = FinQA_prompt.format(pre_text, table, post_text, question, "")
        True_response = program + ' = ' + answer
        texts.append(text)
        responses.append(True_response)
    return { "text" : texts, "true_responses" : responses,}

test = test.map(formatting_prompts_test, batched = True)

In [ ]:
print(test["text"][0])
#print(test["true_responses"][0])

In [ ]:
FastLanguageModel.for_inference(model)

prompt = ["""### pretext:
Can you calculate this: 3 * 97 / 45 + 34?
### Response:
"""]


test_input = tokenizer(prompt, return_tensors = "pt").to("cuda")
outputs = model.generate(**test_input, max_new_tokens = 64, use_cache = True)
tokenizer.batch_decode(outputs)

In [ ]:
FastLanguageModel.for_inference(model)

prompt = ["""### pretext:
Can you explain about what are you able to do?
### Response:
"""]


test_input = tokenizer(prompt, return_tensors = "pt").to("cuda")
outputs = model.generate(**test_input, max_new_tokens = 64, use_cache = True)
tokenizer.batch_decode(outputs)

In [ ]:
import re

pred_test_output = []



def extract_response(text):
    pattern = r"Response:\n(.*?)<\|eot_id\|>"
    match = re.search(pattern, text)
    if match:
        return match.group(1)
    return None


for i in range(len(test["text"])):
    FastLanguageModel.for_inference(model)
    test_input = tokenizer(test["text"][i], return_tensors = "pt").to("cuda")
    outputs = model.generate(**test_input, max_new_tokens = 64, use_cache = True, temperature = 1e-10)
    decoded_output = tokenizer.batch_decode(outputs)
    pred_test_output.append(extract_response(decoded_output[0]))

In [ ]:
true_test_output = test["true_responses"]

In [ ]:
import pickle
with open("pred_test_output.pkl", 'wb') as file:
    pickle.dump(pred_test_output, file)

In [ ]:
import pickle
with open("true_test_output.pkl", 'wb') as file:
    pickle.dump(true_test_output, file)

In [ ]:
import pickle
with open("test_prompts.pkl", 'wb') as file:
    pickle.dump(test["text"], file)

In [ ]:
!pip install rouge-score

In [ ]:
from rouge_score import rouge_scorer

# Initialize the ROUGE scorer
scorer = rouge_scorer.RougeScorer(['rougeL'], use_stemmer=True)

scores = []


scores = []
attribute_error_count = 0

try:
    for ref, hyp in zip(true_test_output, pred_test_output):
        try:
            score = scorer.score(ref, hyp)
            scores.append(score)
        except AttributeError as e:
            print(f"An AttributeError occurred: {e}")
            attribute_error_count += 1
except Exception as e:
    print(f"An unexpected error occurred: {e}")

# Print the total number of AttributeError exceptions
print(f"Total number of AttributeError exceptions: {attribute_error_count}")

    

In [ ]:
suum1 = 0
suum2 = 0
suum3 = 0

for i in range(len(scores)):
    suum1 += float(scores[i]['rougeL'][0])
    suum2 += float(scores[i]['rougeL'][1])
    suum3 += float(scores[i]['rougeL'][2])

precision = suum1 / len(scores)
recall = suum2 / len(scores)
fmeasure = suum3 / len(scores)
print("The Precision (Rouge-L): {0:.2f}".format(precision))
print("The Recall (Rouge-L):    {0:.2f}".format(recall))
print("The F-Measure (Rouge-L): {0:.2f}".format(fmeasure))


In [ ]:
indexes = []
for i in range(len(scores)):
    f_val = float(scores[i]['rougeL'][2])
    if f_val < 0.4:
        indexes.append(i)

print(len(indexes))

In [ ]:
rougeL_fmeasure = []

for i in range(len(scores)):
    f_val = float(scores[i]['rougeL'][2])
    rougeL_fmeasure.append(f_val)

print(rougeL_fmeasure.index(min(rougeL_fmeasure)))

# Loading The Outputs and Metric Reports:

In [ ]:
!pip install rouge-score

In [ ]:
from rouge_score import rouge_scorer

# Initialize the ROUGE scorer
scorer = rouge_scorer.RougeScorer(['rougeL'], use_stemmer=True)

scores = []


scores = []
attribute_error_count = 0

try:
    for ref, hyp in zip(true_test_output, pred_test_output):
        try:
            score = scorer.score(ref, hyp)
            scores.append(score)
        except AttributeError as e:
            print(f"An AttributeError occurred: {e}")
            attribute_error_count += 1
except Exception as e:
    print(f"An unexpected error occurred: {e}")

# Print the total number of AttributeError exceptions
print(f"Total number of AttributeError exceptions: {attribute_error_count}")

    

In [ ]:
import pickle

with open("/kaggle/input/the-finetuned-llama-output/pred_test_output.pkl", 'rb') as file:
    pred_test_output = pickle.load(file)
with open("/kaggle/input/the-finetuned-llama-output/true_test_output.pkl", 'rb') as file:
    true_test_output = pickle.load(file)


In [ ]:
type(pred_test_output)

In [ ]:
n = 675
print("The sample index:                         ",n )
print("The F-measure score for this responce is: ", scores[n]['rougeL'])
print("The true responce is:                     ", true_test_output[n])
print("The model output is:                      ", pred_test_output[n])

In [ ]:
n = 3
print("The sample index:                         ",n )
print("The F-measure score for this responce is: ", scores[n]['rougeL'])
print("The true responce is:                     ", true_test_output[n])
print("The model output is:                      ", pred_test_output[n])

In [ ]:
n = 963
print("The sample index:                         ",n )
print("The F-measure score for this responce is: ", scores[n]['rougeL'])
print("The true responce is:                     ", true_test_output[n])
print("The model output is:                      ", pred_test_output[n])

In [ ]:
n = 594
print("The sample index:                         ",n )
print("The F-measure score for this responce is: ", scores[n]['rougeL'])
print("The true responce is:                     ", true_test_output[n])
print("The model output is:                      ", pred_test_output[n])

In [ ]:
test["text"][594]

In [ ]:
n = 592
print("The sample index:                         ",n )
print("The F-measure score for this responce is: ", scores[n]['rougeL'])
print("The true responce is:                     ", true_test_output[n])
print("The model output is:                      ", pred_test_output[n])

In [ ]:
type(test["text"])